In [ ]:
%run ./initialize.ipynb
import lion_tools
spark=lion_tools.get_or_create_spark()
lion_tools.extend_dataframe()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 13:46:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pyspark.sql.functions as F

def normalize_data(row):
    return {k: float(v) if isinstance(v, int) else v for k, v in row.items()}

def get_dataset(url, name, normalize=False, transformers=[]):
    import ipynbname
    import requests
    import json
    import os

    data = requests.get(url).json()

    if normalize:
        data = [normalize_data(r) for r in data]

    df = spark.createDataFrame(data)

    for transformer in transformers:
        df = transformer(df)

    notebook_path = ipynbname.path()
    home_path = notebook_path.parent.parent
    
    import shutil
    import glob
    
    temp_path = os.path.join(home_path, "datasets", f"temp_{name}")
    df.coalesce(1).write.mode("overwrite").parquet(temp_path)
    parquet_file = glob.glob(os.path.join(temp_path, "*.parquet"))[0]
    final_path = os.path.join(home_path, "datasets", f"{name}.parquet")
    shutil.move(parquet_file, final_path)
    shutil.rmtree(temp_path)

    print(f"Dataset '{name}' saved to {final_path}")




In [3]:
get_dataset(
    "https://raw.githubusercontent.com/vega/vega/master/docs/data/movies.json",
    "movies",
    normalize=True,
    transformers=[
        lambda df: df.withColumn("IMDB Rating", F.col("IMDB Rating").cast("decimal(3,1)")),
        lambda df: df.withColumn("IMDB Votes", F.col("IMDB Votes").cast("integer")),
    ]
)

Dataset 'movies' saved to /Users/micheldeleeuw/dev/lion-tools/datasets/movies.parquet
